In [2]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_community.chat_models import ChatOpenAI
from langchain.chat_models import AzureChatOpenAI
from langchain_openai import AzureOpenAI 
import pandas as pd
import dice_ml
from pathlib import Path


import os

In [3]:
def get_network_activity_info():
    feature_description = {
        'user': 'Unique identifier for a user in the network',
        'logon_on_own_pc_normal': 'Logon event on the user’s assigned PC during normal hours (binary)',
        'logon_on_other_pc_normal': 'Logon event on a different PC during normal hours (binary)',
        'logon_on_own_pc_off_hour': 'Logon event on the user’s assigned PC during off-hours (binary)',
        'logon_on_other_pc_off_hour': 'Logon event on a different PC during off-hours (binary)',
        'logon_hour': 'Hour of the day the logon event occurred (0-23)',
        'day_of_a_week': 'Day of the week when the activity occurred (0=Monday, 6=Sunday)',
        'device_connects_on_own_pc': 'Number of device connections to the user’s assigned PC',
        'device_connects_on_other_pc': 'Number of device connections to a different PC',
        'device_connects_on_own_pc_off_hour': 'Device connections to user’s assigned PC during off-hours',
        'device_connects_on_other_pc_off_hour': 'Device connections to a different PC during off-hours',
        'documents_copy_own_pc': 'Number of documents copied on the user’s assigned PC',
        'documents_copy_other_pc': 'Number of documents copied on a different PC',
        'exe_files_copy_own_pc': 'Number of executable files copied on the user’s assigned PC',
        'exe_files_copy_other_pc': 'Number of executable files copied on a different PC',
        'documents_copy_own_pc_off_hour': 'Documents copied on the user’s assigned PC during off-hours',
        'documents_copy_other_pc_off_hour': 'Documents copied on a different PC during off-hours',
        'exe_files_copy_own_pc_off_hour': 'Executable files copied on the user’s assigned PC during off-hours',
        'exe_files_copy_other_pc_off_hour': 'Executable files copied on a different PC during off-hours',
        'neutral_sites': 'Number of visits to neutral (non-malicious, non-job-search) websites',
        'job_search': 'Number of job search website visits',
        'hacking_sites': 'Number of hacking-related website visits',
        'neutral_sites_off_hour': 'Number of visits to neutral websites during off-hours',
        'job_search_off_hour': 'Number of job search website visits during off-hours',
        'hacking_sites_off_hour': 'Number of hacking-related website visits during off-hours',
        'total_emails': 'Total number of emails sent',
        'int_to_int_mails': 'Number of internal-to-internal emails sent',
        'int_to_out_mails': 'Number of internal-to-external emails sent',
        'out_to_int_mails': 'Number of external-to-internal emails received',
        'out_to_out_mails': 'Number of external-to-external emails sent',
        'internal_recipients': 'Number of unique internal email recipients',
        'external_recipients': 'Number of unique external email recipients',
        'distinct_bcc': 'Number of distinct blind carbon copy (BCC) recipients',
        'mails_with_attachments': 'Number of emails sent with attachments',
        'after_hour_mails': 'Number of emails sent after normal working hours',
        'role': 'User’s role level within the organization',
        'business_unit': 'Business unit the user belongs to',
        'functional_unit': 'Functional unit within the business',
        'department': 'Department the user belongs to',
        'team': 'Team assignment of the user',
        'O': 'Openness personality trait score',
        'C': 'Conscientiousness personality trait score',
        'E': 'Extraversion personality trait score',
        'A': 'Agreeableness personality trait score',
        'N': 'Neuroticism personality trait score'
    }
    return feature_description


In [4]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import tensorflow as tf
from keras import optimizers, Sequential
from keras.layers import Dense, LSTM, RepeatVector, TimeDistributed, Dropout, Input
from keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping
from keras.models import Model, load_model
from keras import regularizers
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix, auc, roc_curve
import os
from tqdm import tqdm

LABELS = ["Normal", "Anomaly"]

### 🟢 Step 1: Data Preprocessing Functions ###
def read_data(path):
    """Reads and preprocesses dataset."""
    df1 = pd.read_csv(path)
    df1 = df1.drop(['start_ts', 'session_duration'], axis=1)
    df1 = df1.fillna(0)
    df1['role'] = df1['role'].astype('category').cat.codes
    df1['user'] = df1['user'].astype('category').cat.codes
    return df1

def get_train_test_data(train, test, lookback):
    """Prepares training and testing sequences."""
    sc = MinMaxScaler(feature_range=(0, 1))
    train_scaled = sc.fit_transform(train.drop(columns=['class', 'type'], errors='ignore'))
    test_scaled = sc.transform(test.drop(columns=['class', 'type'], errors='ignore'))

    # Creating time-series sequences
    X_train = np.array([train_scaled[i - lookback:i, :] for i in range(lookback, len(train_scaled))])
    X_test = np.array([test_scaled[i - lookback:i, :] for i in range(lookback, len(test_scaled))])
    y_test = test.iloc[lookback - 1:, test.columns.get_loc('class')].values

    return X_train, X_test, y_test, sc

### 🟢 Step 2: Counterfactual Generation Function ###

def generate_diverse_counterfactuals(
    model, sequence, scaler, feature_names, num_counterfactuals=5, 
    learning_rate=0.01, iterations=500, threshold=0.04, immutable_features=[], diversity_weight=0.01
):
    """
    Generates diverse counterfactuals while ensuring they are classified as normal 
    (i.e., reconstruction error is below the given threshold). A diversity term is added 
    to the loss to encourage counterfactuals to be different from one another.
    """
    counterfactuals = []
    print("\n🚀 Generating Counterfactuals...\n")

    # Convert the input sequence to a TensorFlow tensor.
    sequence_tf = tf.convert_to_tensor(sequence, dtype=tf.float32)
    
    # Determine indices for immutable features.
    immutable_indices = np.array(
        [feature_names.index(feature) for feature in immutable_features if feature in feature_names], dtype=int
    )
    original_immutable_values = sequence[:, :, immutable_indices].astype(np.float32)

    for cf_idx in tqdm(range(num_counterfactuals), desc="Generating Counterfactuals", unit="cf"):
        # Initialize the candidate counterfactual with the original sequence.
        seq_cf = tf.Variable(sequence_tf, dtype=tf.float32)
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

        for _ in tqdm(range(iterations), desc=f"Optimizing CF {cf_idx+1}/{num_counterfactuals}", unit="step", leave=False):
            with tf.GradientTape() as tape:
                tape.watch(seq_cf)
                reconstruction = model(seq_cf)
                reconstruction_loss = tf.reduce_mean(tf.abs(reconstruction - seq_cf))
                
                # Apply a penalty if reconstruction error is above the threshold.
                penalty = tf.maximum(reconstruction_loss - threshold, 0) * 10.0

                # Compute diversity loss: for each already finalized counterfactual,
                # penalize if the current candidate is too similar.
                diversity_loss = 0.0
                if len(counterfactuals) > 0:
                    for prev in counterfactuals:
                        prev_tensor = tf.convert_to_tensor(prev, dtype=tf.float32)
                        distance = tf.norm(seq_cf - prev_tensor)
                        diversity_loss += 1.0 / (distance + 1e-8)  # Avoid division by zero

                # Total loss includes reconstruction loss, anomaly penalty, and diversity term.
                total_loss = reconstruction_loss + penalty + diversity_weight * diversity_loss

            grads = tape.gradient(total_loss, seq_cf)
            if grads is not None:
                grads_numpy = grads.numpy()
                # Zero out gradients for immutable features so they remain unchanged.
                if len(immutable_indices) > 0:
                    grads_numpy[:, :, immutable_indices] = 0  
                grads_tf = tf.convert_to_tensor(grads_numpy, dtype=tf.float32)
                optimizer.apply_gradients([(grads_tf, seq_cf)])

                # Clip the candidate values to [0, 1].
                seq_cf.assign(tf.clip_by_value(seq_cf, 0, 1))

                # Restore immutable features using their original values.
                updates = tf.convert_to_tensor(original_immutable_values, dtype=tf.float32)
                indices = np.array([[b, t, f] 
                                     for b in range(seq_cf.shape[0]) 
                                     for t in range(seq_cf.shape[1]) 
                                     for f in immutable_indices])
                seq_cf.assign(tf.tensor_scatter_nd_update(seq_cf, indices, tf.reshape(updates, [-1])))

            # Early stopping if the candidate's reconstruction error is below the threshold.
            if reconstruction_loss.numpy() < threshold:
                print(f"✅ CF {cf_idx+1} is now normal (MSE={reconstruction_loss.numpy():.5f}) - stopping early")
                break

        counterfactuals.append(seq_cf.numpy())

    print("\n✅ Counterfactual Generation Complete!")

    # Reshape and inverse-transform to get the counterfactuals back on the original scale.
    counterfactuals = np.array(counterfactuals)
    reshaped_cf = counterfactuals.reshape(-1, sequence.shape[-1])
    counterfactuals_original_scale = scaler.inverse_transform(reshaped_cf)
    counterfactuals_original_scale = counterfactuals_original_scale.reshape(num_counterfactuals, sequence.shape[1], sequence.shape[2])

    return [pd.DataFrame(cf, columns=feature_names) for cf in counterfactuals_original_scale]



### 🟢 Step 3: Model Evaluation Function ###
def evaluate_model_on_sequences(model, anomalous_sequence, counterfactual_sequences, feature_names, scaler):
    """
    Evaluates both the original anomaly and generated counterfactuals.
    """
    anomaly_reconstructed = model.predict(anomalous_sequence)
    anomaly_error = np.mean(np.abs(anomalous_sequence - anomaly_reconstructed))

    evaluation_results = []
    for cf_sequence in counterfactual_sequences:
        cf_sequence = scaler.transform(cf_sequence)
        cf_sequence = cf_sequence.reshape(1, anomalous_sequence.shape[1], anomalous_sequence.shape[2])
        cf_reconstructed = model.predict(cf_sequence)
        cf_error = np.mean(np.abs(cf_sequence - cf_reconstructed))

        anomaly_original = scaler.inverse_transform(anomalous_sequence.reshape(-1, len(feature_names)))
        counterfactual_original = scaler.inverse_transform(cf_sequence.reshape(-1, len(feature_names)))
        feature_differences = np.abs(anomaly_original - counterfactual_original)
        diff_df = pd.DataFrame(feature_differences, columns=feature_names)
        mean_differences = diff_df.mean().sort_values(ascending=False)
        sorted_feature_differences = pd.DataFrame(mean_differences, columns=['Difference'])

        evaluation_results.append({
            "counterfactual_reconstruction_error": cf_error,
            "feature_differences": sorted_feature_differences
        })

    print("\n🔍 **Model Evaluation Results:**")
    print(f"🛑 Anomalous Session Reconstruction Error: {anomaly_error:.6f}")

    for idx, result in enumerate(evaluation_results):
        print(f"\n✅ Counterfactual {idx+1} Reconstruction Error: {result['counterfactual_reconstruction_error']:.6f}")
        print(result["feature_differences"].to_string())
    return evaluation_results

### 🟢 Step 4: Main Execution ###

path = "/home/sathish/UEBA/data/data.csv"
df = read_data(path)
train_data, test_data = df.iloc[:276388], df.iloc[276388:]
lookback = 3
X_train, X_test, y_test, scaler = get_train_test_data(train_data, test_data, lookback)
n_features = X_train.shape[2]

# Load or train model
model_path = "lstm_autoencoder.h5"
lstm_model = load_model(model_path, custom_objects={'MeanSquaredError': tf.keras.losses.MeanSquaredError()}) if os.path.exists(model_path) else None

mse = np.mean(np.power(X_test - lstm_model.predict(X_test), 2), axis=(1, 2))
threshold = 0.04
anomaly_idx = np.where(mse > threshold)[0][0]
anomalous_sequence = X_test[anomaly_idx].reshape(1, lookback, X_train.shape[2])

feature_names = train_data.drop(columns=['class', 'type'], errors='ignore').columns.tolist()
immutable_features=["user", "role", "O", "C", "E", "A", "N"]
# Set threshold for counterfactual generation to 0.02
counterfactual_examples = generate_diverse_counterfactuals(
    lstm_model,
    anomalous_sequence,
    scaler,
    feature_names,
    threshold=0.03,
    immutable_features=immutable_features
)
evaluate_model_on_sequences(lstm_model, anomalous_sequence, counterfactual_examples, feature_names, scaler)




2025-03-11 09:08:02.319656: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-11 09:08:02.323179: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-11 09:08:02.333718: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741698482.352356 1416949 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741698482.357880 1416949 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-11 09:08:02.377262: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

2045/2045 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step

🚀 Generating Counterfactuals...



Generating Counterfactuals:  20%|██        | 1/5 [00:02<00:08,  2.18s/cf]

✅ CF 1 is now normal (MSE=0.02985) - stopping early


Generating Counterfactuals:  40%|████      | 2/5 [00:04<00:07,  2.49s/cf]

...

✅ CF 4 is now normal (MSE=0.02990) - stopping early


Generating Counterfactuals: 100%|██████████| 5/5 [00:16<00:00,  3.32s/cf]

✅ CF 5 is now normal (MSE=0.02975) - stopping early

✅ Counterfactual Generation Complete!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step

🔍 **Model Evaluation Results:**
🛑 Anomalous Session Reconstruction Error: 0.085482

✅ Counterfactual 1 Reconstruction Error: 0.030234
                                      Difference
neutral_sites                          12.369064
internal_recipients                     6.458809
documents_copy_own_pc                   5.734789
external_recipients                     4.434732
documents_copy_own_pc_off_hour          3.984578
int_to_int_mails                        3.292809
neutral_sites_off_hour                  2.978429
out_to_out_mails                        2.674841
mails_with_attachments                  1.380025
job_search                              1.319440
logon_on_own_pc_normal                  0.864271
distinct_bcc                            0.826717
device_connects_on_own_pc_off_hour    

[{'counterfactual_reconstruction_error': 0.030233892,
  'feature_differences':                                       Difference
  neutral_sites                          12.369064
  internal_recipients                     6.458809
  documents_copy_own_pc                   5.734789
  external_recipients                     4.434732
  documents_copy_own_pc_off_hour          3.984578
  int_to_int_mails                        3.292809
  neutral_sites_off_hour                  2.978429
  out_to_out_mails                        2.674841
  mails_with_attachments                  1.380025
  job_search                              1.319440
  logon_on_own_pc_normal                  0.864271
  distinct_bcc                            0.826717
  device_connects_on_own_pc_off_hour      0.700924
  team                                    0.697029
  day_of_a_week                           0.657302
  total_emails                            0.629372
  logon_hour                              0.566246
  log

In [5]:
import json

def format_data_for_llm(anomalous_sequence, counterfactuals, feature_names, scaler):
    # Convert anomalous sequence to original scale
    anomalous_original = scaler.inverse_transform(anomalous_sequence.reshape(-1, len(feature_names)))

    # Convert counterfactual DataFrames to NumPy arrays does not need scaling as it is already scaled correctly
    counterfactuals_original = [
        cf_df.to_numpy().astype(float).reshape(-1, len(feature_names))  # Convert to float64
        for cf_df in counterfactuals  # Convert DataFrame to NumPy array
    ]

    # Convert to list of dictionaries with rounded values (ensuring float conversion)
    anomalous_session = [
        {feature_names[i]: round(float(value), 2) for i, value in enumerate(row)}
        for row in anomalous_original
    ]

    counterfactuals_list = [
        [
            {feature_names[i]: round(float(value), 2) for i, value in enumerate(row)}
            for row in cf_seq
        ]
        for cf_seq in counterfactuals_original
    ]

    # Create dictionary for LLM input
    formatted_data = {
        "anomalous_session": anomalous_session,
        "counterfactuals": counterfactuals_list
    }

    # Convert to JSON string (ensure_ascii=False to support special characters)
    return json.dumps(formatted_data, indent=4, ensure_ascii=False)


In [6]:
# 🟢 NEW: Format the anomaly + counterfactuals for LLM input
formatted_data = format_data_for_llm(anomalous_sequence, counterfactual_examples, feature_names, scaler)

# 🟢 Print or save the JSON data
print("\n🔹 **Formatted Data for LLM:**")
print(formatted_data)  # Print to console



🔹 **Formatted Data for LLM:**
{
    "anomalous_session": [
        {
            "user": 675.0,
            "logon_on_own_pc_normal": 0.0,
            "logon_on_other_pc_normal": 0.0,
            "logon_on_own_pc_off_hour": 1.0,
            "logon_on_other_pc_off_hour": 0.0,
            "logon_hour": 7.0,
            "day_of_a_week": 1.0,
            "device_connects_on_own_pc": 7.0,
            "device_connects_on_other_pc": 0.0,
            "device_connects_on_own_pc_off_hour": 0.0,
            "device_connects_on_other_pc_off_hour": 0.0,
            "documents_copy_own_pc": 22.0,
            "documents_copy_other_pc": 0.0,
            "exe_files_copy_own_pc": 0.0,
            "exe_files_copy_other_pc": 0.0,
            "documents_copy_own_pc_off_hour": 0.0,
            "documents_copy_other_pc_off_hour": 0.0,
            "exe_files_copy_own_pc_off_hour": 0.0,
            "exe_files_copy_other_pc_off_hour": 0.0,
            "neutral_sites": 89.0,
            "job_search": 0.0,
     

In [7]:
#print(anomalous_sequence.shape)

co_original = scaler.inverse_transform(anomalous_sequence.reshape(-1, len(feature_names)))
#print(co_original, type(co_original))
#print(co_original[0])
co_original_df = pd.DataFrame(co_original.reshape(3,-1),columns=counterfactual_examples[0].columns)
print(co_original_df.to_string())

    user  logon_on_own_pc_normal  logon_on_other_pc_normal  logon_on_own_pc_off_hour  logon_on_other_pc_off_hour  logon_hour  day_of_a_week  device_connects_on_own_pc  device_connects_on_other_pc  device_connects_on_own_pc_off_hour  device_connects_on_other_pc_off_hour  documents_copy_own_pc  documents_copy_other_pc  exe_files_copy_own_pc  exe_files_copy_other_pc  documents_copy_own_pc_off_hour  documents_copy_other_pc_off_hour  exe_files_copy_own_pc_off_hour  exe_files_copy_other_pc_off_hour  neutral_sites  job_search  hacking_sites  neutral_sites_off_hour  job_search_off_hour  hacking_sites_off_hour  total_emails  int_to_int_mails  int_to_out_mails  out_to_int_mails  out_to_out_mails  internal_recipients  external_recipients  distinct_bcc  mails_with_attachments  after_hour_mails  role  business_unit  functional_unit  department  team     O     C     E     A     N
0  675.0                     0.0                       0.0                       1.0                         0.0         

In [8]:
#cf_example = pd.DataFrame(counterfactual_examples[0].iloc[0])
cf_example = counterfactual_examples
#cf_example1  = cf_example.T.round(1)
print(type(cf_example))
cf_example[4]   # list of CF dataframes


<class 'list'>


,user,logon_on_own_pc_normal,logon_on_other_pc_normal,logon_on_own_pc_off_hour,logon_on_other_pc_off_hour,logon_hour,day_of_a_week,device_connects_on_own_pc,device_connects_on_other_pc,device_connects_on_own_pc_off_hour,...,role,business_unit,functional_unit,department,team,O,C,E,A,N
0,675.0,0.713983,0.007962,0.535136,0.0,7.441973,1.977825,7.373900,0.015285,0.294130,...,14.0,1.0,2.837465,2.099065,3.843745,39.0,40.0,47.0,50.0,26.0
1,675.0,0.854455,0.014584,0.401808,0.0,7.579709,2.011969,7.277054,0.001431,0.266381,...,14.0,1.0,2.961633,2.349903,3.085686,39.0,40.0,47.0,50.0,26.0
2,675.0,0.837595,0.000000,0.403395,0.0,7.557063,1.978618,7.033497,0.010309,0.221397,...,14.0,1.0,2.922812,2.143360,2.789212,39.0,40.0,47.0,50.0,26.0


In [9]:
 # Initialize Azure OpenAI client
llm = AzureChatOpenAI(
            deployment_name="gpt-4o",
            openai_api_base=os.getenv("ENDPOINT_URL", "https://g4266-m3cff6ws-swedencentral.openai.azure.com/"),
            openai_api_key=os.getenv("AZURE_OPENAI_API_KEY", "6xxmkjbfwnm3Li1keZmIG0V5cggGUXPqfSdAppx2EkW0iUSXSbrDJQQJ99AKACfhMk5XJ3w3AAAAACOGdLLq"),
            openai_api_version="2024-05-01-preview"
        )


/tmp/ipykernel_1416949/2128844656.py:2: LangChainDeprecationWarning: The class `AzureChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import AzureChatOpenAI``.
  llm = AzureChatOpenAI(
/home/sathish/UEBA/.venv/lib/python3.10/site-packages/langchain_community/chat_models/azure_openai.py:174: UserWarning: As of openai>=1.0.0, Azure endpoints should be specified via the `azure_endpoint` param not `openai_api_base` (or alias `base_url`). Updating `openai_api_base` from https://g4266-m3cff6ws-swedencentral.openai.azure.com/ to https://g4266-m3cff6ws-swedencentral.openai.azure.com/openai.
  warnings.warn(
/home/sathish/UEBA/.venv/lib/python3.10/site-packages/langchain_community/chat_models/azure_openai.py:181: UserWarning: As of openai>=1.0.0, if `deployment_

In [10]:
def ZeroShotExplain1():
    return  """
        Im providing a negative outcome from a {ML_system} and your task provide obeservations over the provided Insider Anomaly flagged by a LSTM model by comparing them to Counterfactuals generated from the Anomaly. 

        ----- Anomolous outcome -----
        {negative_outcome}

        
        ----- Positive couterfactual outcome -----
        {positive_outcome}


        ----- Observations -----
        <List of Observations>
        """
def ZeroShotRules():
    return  """
        Im providing a negative outcome from a {ML_system} and your task is to extract the most important observed rules over the provided Insider Anomaly flagged by a LSTM model by comparing them to Counterfactuals generated from the Anomaly. 

        --------Immutable features across Anomaly and Counterfactuals ----
        {immutable_features}

        
        ----- Anomolous outcome -----
        {negative_outcome}

        ----- Positive couterfactual outcome -----
        {positive_outcome}

        ----- Rules -----
        <List of Rules>
        """
def ZeroShotRulesCode():
    return  """
        Im providing a negative outcome from a {ML_system}, a set of counterfactual cases that flip the decision of the system and the main rules inferred from the counterfactuals.
        You should generate python code to count how many of the counterfactuals are consistent with the rule. The code should create a df with the counterfactuals provided and then check for each rule how many of them follow the rules. Order the rules. Finally, you should print the results. 
        Keep the original structure of the negative_outcome and positive_outcome when you generate the python code.

        --------Immutable features across Anomaly and Counterfactuals ----
        {immutable_features}
 
        ----- Anomolous outcome -----
        {negative_outcome}

        ----- Positive couterfactual outcome -----
        {positive_outcome}

        ----- Rules -----
        {rules}
        
        ----- Dataset info -----
        The following info about the dataset is available:
        {dataset_info}
        
        ----- Code -----
        ```
        import pandas as pd
        #complete this code
        ```
        """ 

In [11]:

result = ZeroShotExplain1()
print(result)


        Im providing a negative outcome from a {ML_system} and your task provide obeservations over the provided Insider Anomaly flagged by a LSTM model by comparing them to Counterfactuals generated from the Anomaly. 

        ----- Anomolous outcome -----
        {negative_outcome}

        
        ----- Positive couterfactual outcome -----
        {positive_outcome}


        ----- Observations -----
        <List of Observations>
        


In [12]:
template0 = ZeroShotExplain1()
template1 = ZeroShotRules()
template2 = ZeroShotRulesCode()
#template3 = ZeroShotExplanation(user_input=False)
#template4 = ZeroShotExample()
#template5 = ZeroShotExampleCode()

In [13]:
model_description = """ML_system that detects Anomolous behavior on a network"""

In [14]:
import tiktoken
tokenizer = tiktoken.encoding_for_model("gpt-4")
# Extract the prompt text
# ✅ Ensure co_original_df and cf_example[4] are properly converted to strings
negative_outcome = co_original_df.to_string()  # Ensure calling .to_string()
positive_outcome = cf_example[4].to_string()
# ✅ Properly define the prompt template
prompt = PromptTemplate(
    input_variables=["ML_system", "negative_outcome", "positive_outcome"],
    template=template0
)

formatted_prompt = prompt.format(   
    ML_system="ML_system that detects Anomalous behavior on a network",
    negative_outcome=negative_outcome,
    positive_outcome=positive_outcome)
# ✅ Tokenize and count tokens
tokenizer = tiktoken.encoding_for_model("gpt-4")
token_count = len(tokenizer.encode(formatted_prompt))

print(f"Token count for this GPT-4 request: {token_count}")

Token count for this GPT-4 request: 1985


In [15]:
rules_chain = LLMChain(
            llm=llm,
            prompt=PromptTemplate(
                input_variables=["ML_system", "negative_outcome", "positive_outcome"],
                template=template0
            )
        )

/tmp/ipykernel_1416949/2425814101.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  rules_chain = LLMChain(


In [31]:
type(cf_example)

list

In [ ]:
###Converting to straight text 
negative_outcome_str = co_original_df.to_string(index=False) 
positive_outcome_str = "\n\n".join([df.to_string(index=False) for df in cf_example])
response = rules_chain.run({
    "ML_system": model_description,
    "immutable_features": immutable_features,
    "negative_outcome": negative_outcome_str,  # Pass as string
    "positive_outcome": positive_outcome_str   # Pass as string
})

In [35]:
response
sections = response.split("\n\n")  # Split by paragraph

# Print each section with a clear heading
for i, section in enumerate(sections, start=1):
    print(f"\n🔹 Section {i}:\n{section}")


🔹 Section 1:
To extract the most important observed rules, we need to compare the anomalous data with the counterfactuals and identify the features or patterns that significantly differ between the two groups. These differences highlight the key factors contributing to the anomalous behavior. Below is a structured analysis based on the provided data:

🔹 Section 2:
---

🔹 Section 3:
### **Important Observed Rules for the Insider Anomaly (Based on Feature Comparisons)**

🔹 Section 4:
#### **1. Logon Patterns**
   - **Feature:** `logon_on_own_pc_normal`  
     - **Anomalous Outcome:** Always `0.0` (user never logs on their own PC during normal hours).  
     - **Counterfactuals:** Values range from `0.712926` to `0.961193` (higher logon activity on own PC during normal hours).  
     - **Rule:** Lack of normal logon activity on the user's own PC is indicative of anomalous behavior.
   - **Feature:** `logon_on_own_pc_off_hour`  
     - **Anomalous Outcome:** Always `1.0` (frequent logons 

In [16]:
response = rules_chain.run(({
        "ML_system": model_description,
        "negative_outcome": co_original_df.to_string,
        "positive_outcome": cf_example[4].to_string
        }))


/tmp/ipykernel_1416949/901859083.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = rules_chain.run(({


In [ ]:
print(response)

In [18]:
# ✅ Ensure `formatted_data` is a dictionary
if isinstance(formatted_data, str):  # If it's a JSON string, parse it
    formatted_data = json.loads(formatted_data)

# ✅ Extract the anomaly and counterfactuals
negative_outcome = formatted_data["anomalous_session"]   # Extract Anomaly
positive_outcome = formatted_data["counterfactuals"]     # Extract Counterfactuals

# ✅ Convert to JSON-formatted strings for better readability
negative_outcome = json.dumps(negative_outcome, indent=4)
positive_outcome = json.dumps(positive_outcome, indent=4)

# ✅ Setup LLM prompt template
template0 = ZeroShotExplain1()
model_description = "ML_system that detects anomalous behavior on a network."

rules_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate(
        input_variables=["ML_system","immutable_features" "negative_outcome", "positive_outcome"],
        template=template1
    )
)
###########Get Token Lenth
formatted_prompt = prompt.format(   
    ML_system=model_description,
    immutable_features=immutable_features,
    negative_outcome=negative_outcome,
    positive_outcome=positive_outcome,
    )
# ✅ Tokenize and count tokens
tokenizer = tiktoken.encoding_for_model("gpt-4")
token_count = len(tokenizer.encode(formatted_prompt))

print(f"Token count for this GPT-4 request: {token_count}")

####################
# ✅ Execute the LLM call
explanation = rules_chain.run(
    {
        "ML_system": model_description,
        "immutable_features": immutable_features,
        "negative_outcome": negative_outcome,
        "positive_outcome": positive_outcome
    }
)

# ✅ Print the response
print("\n🔹 **LLM Explainability Response:**")
print(explanation)

Token count for this GPT-4 request: 9848



🔹 **LLM Explainability Response:**
To identify the most important observed rules behind the anomalous behavior flagged by the LSTM model, we need to compare the anomalous outcome data against the counterfactual examples. Counterfactuals are used to explain what changes would have flipped the outcome to a non-anomalous (positive) state. The key is to identify features and patterns that deviate significantly between the anomalous and counterfactual data.

Here’s a step-by-step breakdown:

---

### **Step 1: Immutable Features**
The following features are immutable across both anomalous and counterfactual outcomes:
- `user`
- `role`
- `O`, `C`, `E`, `A`, `N` (personality traits)

These features are fixed and should not contribute to the anomaly detection process.

---

### **Step 2: Features with Significant Deviations**
By comparing the anomalous outcomes with positive counterfactuals, we observe the following significant differences in feature values:

#### **1. Logon Behavior**
   - *

In [19]:
rulescode_chain = LLMChain(
            llm=llm,
            prompt=PromptTemplate(
                input_variables=["ML_system", "immutable_features", "negative_outcome", "positive_outcome", "rules", "dataset_info"],
                template=template2
            )
        )

In [20]:
dataset_info = get_network_activity_info()

In [21]:
#dataset_info

In [22]:
rules = explanation
response = rulescode_chain.run(
    {
        "ML_system": model_description,
        "immutable_features": immutable_features,
        "negative_outcome": negative_outcome,
        "positive_outcome": positive_outcome,
        "rules": rules,
        "dataset_info": dataset_info
    }
)

In [23]:
print(response)

Here is the completed Python code to process the anomalous and counterfactual data, apply the rules, and count how many of the counterfactuals are consistent with each rule:

```python
import pandas as pd

# Anomalous outcome
anomalous_data = [
    {
        "user": 675.0,
        "logon_on_own_pc_normal": 0.0,
        "logon_on_other_pc_normal": 0.0,
        "logon_on_own_pc_off_hour": 1.0,
        "logon_on_other_pc_off_hour": 0.0,
        "logon_hour": 7.0,
        "day_of_a_week": 1.0,
        "device_connects_on_own_pc": 7.0,
        "device_connects_on_other_pc": 0.0,
        "device_connects_on_own_pc_off_hour": 0.0,
        "device_connects_on_other_pc_off_hour": 0.0,
        "documents_copy_own_pc": 22.0,
        "documents_copy_other_pc": 0.0,
        "exe_files_copy_own_pc": 0.0,
        "exe_files_copy_other_pc": 0.0,
        "documents_copy_own_pc_off_hour": 0.0,
        "documents_copy_other_pc_off_hour": 0.0,
        "exe_files_copy_own_pc_off_hour": 0.0,
        "exe_fi

In [24]:
def extract_code(text):
    start_code = text.find("```python") + len("```python\n")
    end_code = text.find("```", start_code)
    return text[start_code:end_code].strip()


In [25]:
code = extract_code(response)
code

'import pandas as pd\n\n# Anomalous outcome\nanomalous_data = [\n    {\n        "user": 675.0,\n        "logon_on_own_pc_normal": 0.0,\n        "logon_on_other_pc_normal": 0.0,\n        "logon_on_own_pc_off_hour": 1.0,\n        "logon_on_other_pc_off_hour": 0.0,\n        "logon_hour": 7.0,\n        "day_of_a_week": 1.0,\n        "device_connects_on_own_pc": 7.0,\n        "device_connects_on_other_pc": 0.0,\n        "device_connects_on_own_pc_off_hour": 0.0,\n        "device_connects_on_other_pc_off_hour": 0.0,\n        "documents_copy_own_pc": 22.0,\n        "documents_copy_other_pc": 0.0,\n        "exe_files_copy_own_pc": 0.0,\n        "exe_files_copy_other_pc": 0.0,\n        "documents_copy_own_pc_off_hour": 0.0,\n        "documents_copy_other_pc_off_hour": 0.0,\n        "exe_files_copy_own_pc_off_hour": 0.0,\n        "exe_files_copy_other_pc_off_hour": 0.0,\n        "neutral_sites": 89.0,\n        "job_search": 0.0,\n        "hacking_sites": 0.0,\n        "neutral_sites_off_hour": 6

In [26]:
import subprocess
import sys
def execute_code(code, timeout=10):
    with open("temp_code.py", "w") as f:
        f.write(code)
    try:
        result = subprocess.run([sys.executable, "temp_code.py"], capture_output=True, text=True, check=True, timeout=timeout)
        return result.stdout, False
    except subprocess.CalledProcessError as e:
        return e.stdout + e.stderr, True
    except subprocess.TimeoutExpired:
        return "Execution timed out.", True
   

In [27]:
result, _ = execute_code(code)

print(result)

Number of counterfactuals consistent with each rule:
Logon Behavior: 1
Device Connections: 1
Email Patterns: 1
Website Browsing: 1
Document Activity: 0

